## The Problem

As I understand it, the CA Department of Education has made a publicly searchable database of LEA reports here:

http://www2.cde.ca.gov/ccs/searchsurvey.aspx

It is very bad for automated data collection, so we will try to get the data out with selenium (a tool to script common browsers - needed in this case due to the extensive use of Javascript in the page).

## Get our list of schools (and districts)

In [1]:
import pandas as pd

In [2]:
school_df = pd.read_excel('./pubschls.xls') # I got this file from Claudia

In [4]:
# These are the base for our searches
school_df[['District', 'School']]

,District,School
0,Alameda County Office of Education,NaN
1,Alameda County Office of Education,FAME Public Charter
2,Alameda County Office of Education,Envision Academy for Arts & Technology
3,Alameda County Office of Education,Aspire California College Preparatory Academy
4,Alameda County Office of Education,Community School for Creative Education
5,Alameda County Office of Education,Yu Ming Charter
6,Alameda County Office of Education,Urban Montessori Charter
7,Alameda County Office of Education,"Technical, Agricultural & Nat."
8,Alameda County Office of Education,Alameda County Juvenile Hall/Court
9,Alameda County Office of Education,Alameda County Community


## Noodling around with Selenium

In [6]:
first_try = school_df.loc[0, 'District']
first_try

'Alameda County Office of Education'

In [70]:
from itertools import count
from time import sleep

# Sadly, I had to pip install this (it's not readily available in conda / binstar AFAICT)
from selenium import webdriver
# This isn't really documented how to import... seems clunky
from selenium.webdriver.common.keys import Keys

In [76]:
driver = webdriver.Firefox()

In [42]:
driver.window_handles

['{d7c5c95f-e1e5-1543-865a-52de272b669f}']

In [77]:
driver.get("http://www2.cde.ca.gov/ccs/searchsurvey.aspx")

In [78]:
# uncheck the checkbox for LEAs without submissions
checkbox = driver.find_element_by_id("ctl00_body750px_ckShowAll")
checkbox.click()

In [74]:
def get_the_window(name):
    searchbox = driver.find_element_by_id("ctl00_body750px_txtPrefix")
    searchbox.clear()
    searchbox.send_keys(name)
    # This was a guess for how long to wait - seems to work for now
    # I'm not smart enough to figure out the id or other info about the box that pops up - you can't right click it
    sleep(2)
    searchbox.send_keys(Keys.ARROW_DOWN)
    searchbox.send_keys(Keys.ENTER)

In [58]:
def save_the_window(name):
    '''Save the HTML in the new window (if there is one)
    
    name : str
        Filename for HTML'''
    try:
        new_window = driver.window_handles[1]
        driver.switch_to_window(new_window)
        with open(name, 'w') as ofile:
            ofile.write(driver.page_source)
        driver.close()
        # We switch back to our main window
        driver.switch_to_window(driver.window_handles[0])
    except IndexError:
        # We didn't find a new window
        pass

In [79]:
for i, district, school in zip(count(), school_df.District, school_df.School):
    if i % 10 == 0:
        print(i)
    if pd.isnull(school):
        to_search = district
    else:
        to_search = school
        
    get_the_window(to_search)
    save_the_window(to_search + '.html')       

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

NoSuchElementException: Message: Unable to locate element: {"method":"id","selector":"ctl00_body750px_txtPrefix"}
Stacktrace:
    at FirefoxDriver.prototype.findElementInternal_ (file:///var/folders/bp/vm5h3k411wg_f8gdsrrklfnm0000gn/T/tmp6c_05wel/extensions/fxdriver@googlecode.com/components/driver-component.js:10271)
    at FirefoxDriver.prototype.findElement (file:///var/folders/bp/vm5h3k411wg_f8gdsrrklfnm0000gn/T/tmp6c_05wel/extensions/fxdriver@googlecode.com/components/driver-component.js:10280)
    at DelayedCommand.prototype.executeInternal_/h (file:///var/folders/bp/vm5h3k411wg_f8gdsrrklfnm0000gn/T/tmp6c_05wel/extensions/fxdriver@googlecode.com/components/command-processor.js:12274)
    at DelayedCommand.prototype.executeInternal_ (file:///var/folders/bp/vm5h3k411wg_f8gdsrrklfnm0000gn/T/tmp6c_05wel/extensions/fxdriver@googlecode.com/components/command-processor.js:12279)
    at DelayedCommand.prototype.execute/< (file:///var/folders/bp/vm5h3k411wg_f8gdsrrklfnm0000gn/T/tmp6c_05we

In [72]:
%debug

> /opt/anaconda/envs/py3/lib/python3.4/site-packages/selenium/webdriver/remote/webelement.py(316)send_keys()
    315             else:
--> 316                 for i in range(len(val)):
    317                     typing.append(val[i])

ipdb> up
> <ipython-input-59-f835e62db608>(3)get_the_window()
      2     searchbox = driver.find_element_by_id("ctl00_body750px_txtPrefix")
----> 3     searchbox.send_keys(name)
      4     # This was a guess for how long to wait - seems to work for now

ipdb> name
nan
ipdb> q


AttributeError: 'NoneType' object has no attribute 'set_completer_delims'

In [64]:
row = school_df.loc[0, ['District', 'School']]

In [68]:
pd.isnull(row.School)

True